In [1]:
import sys
import os
#!pip uninstall -y websocket-client    
try:
    import websocket
except ImportError:
    !{sys.executable} -m pip  install websocket-client --user #should import succesfully without restarting the kernel/script
    import websocket

import json
import time
from pprint import pprint as pp
import copy
import http.client
import io
from wikidata.client import Client
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.request
import pandas as pd
import pyowm
from datetime import datetime as dt
from operator import itemgetter
import requests
from bs4 import BeautifulSoup
from langdetect import detect
from googletrans import Translator
import re
import emoji
from arango import ArangoClient
from rdflib import Graph, plugin, URIRef, Literal, ConjunctiveGraph, store, Namespace
from rdflib.plugin import register, Parser
#!pip install rdflib_jsonld --user
import rdflib_jsonld
#!pip install pyld --user
from pyld import jsonld
import joblib as jl
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import numpy as np

In [2]:
register('json-ld', Parser, 'rdflib_jsonld.parser', 'JsonLDParser')

<h1 align = "center" style = "color:darkred">WARNING</h1>
<h4 align = "center" style = "color:green">enable UniMiB VPN before running</h4>

## Matching categories with Curlie

In [3]:
meet_to_curl = {
    "arts/culture":"https://curlie.org/Arts/",
    "book clubs":"https://curlie.org/Reference/Books",
    "career/business":"https://curlie.org/Business/",
    "cars/motorcycles":"https://curlie.org/Business/Automotive/",
    "community/environment":"https://curlie.org/Society/Issues/Environment/Organizations/",
    "dancing":"https://curlie.org/Arts/Performing_Arts/Dance",
    "education/learning":"https://curlie.org/Reference/Education/",
    "fashion/beauty":["https://curlie.org/Business/Design/Fashion",
                       "https://curlie.org/Shopping/Health/Beauty"],
    "fitness":"https://curlie.org/Health/Fitness/",
    "food/drink":"https://curlie.org/Recreation/Food/",
    "games":"https://curlie.org/Games/",
    "health/wellbeing":"https://curlie.org/Health/",
    "hobbies/crafts":"https://curlie.org/Shopping/Crafts/",
    "LGBT":"https://curlie.org/Society/Gay%2C_Lesbian%2C_and_Bisexual/",
    "language/ethnic identity":"https://curlie.org/World/",
    "alternative lifestyle":"https://curlie.org/Society/Lifestyle_Choices/",
    "movements/politics":"https://curlie.org/Society/Politics/",
    "movies/film":"https://curlie.org/Arts/Movies/",
    "music":"https://curlie.org/Arts/Music/",
    "new age/spirituality":"https://curlie.org/Society/Religion_and_Spirituality/New_Age/",
    "outdoors/adventure":"https://curlie.org/Recreation/Outdoors/",
    "paranormal":"https://curlie.org/Society/Paranormal/",
    "parents/family":"https://curlie.org/Home/Family/",
    "pets/animals":"https://curlie.org/Recreation/Pets/",
    "photography":"https://curlie.org/Arts/Photography/",
    "religion/beliefs":"https://curlie.org/Society/Religion_and_Spirituality",
    "sci-fi/fantasy":"https://curlie.org/Shopping/Visual_Arts/Thematic/Science_Fiction_and_Fantasy/",
    "singles":"https://curlie.org/Society/People/Singles/",
    "socializing":"https://curlie.org/Society/Relationships/",
    "sports/recreation":"https://curlie.org/Sports/",
    "support":"https://curlie.org/Society/Support_Groups/",
    "tech":"https://curlie.org/Computers/",
    "writing":"https://curlie.org/Arts/Literature/"
}

In [4]:
dmoz_cat={}
for key, item in meet_to_curl.items():
    
    new_key=key.replace("/", " ")
    if key=="arts/culture":
        new_key="fine arts culture"
    dmoz_cat[new_key]=item
dmoz_cat

{'fine arts culture': 'https://curlie.org/Arts/',
 'book clubs': 'https://curlie.org/Reference/Books',
 'career business': 'https://curlie.org/Business/',
 'cars motorcycles': 'https://curlie.org/Business/Automotive/',
 'community environment': 'https://curlie.org/Society/Issues/Environment/Organizations/',
 'dancing': 'https://curlie.org/Arts/Performing_Arts/Dance',
 'education learning': 'https://curlie.org/Reference/Education/',
 'fashion beauty': ['https://curlie.org/Business/Design/Fashion',
  'https://curlie.org/Shopping/Health/Beauty'],
 'fitness': 'https://curlie.org/Health/Fitness/',
 'food drink': 'https://curlie.org/Recreation/Food/',
 'games': 'https://curlie.org/Games/',
 'health wellbeing': 'https://curlie.org/Health/',
 'hobbies crafts': 'https://curlie.org/Shopping/Crafts/',
 'LGBT': 'https://curlie.org/Society/Gay%2C_Lesbian%2C_and_Bisexual/',
 'language ethnic identity': 'https://curlie.org/World/',
 'alternative lifestyle': 'https://curlie.org/Society/Lifestyle_Choic

In [5]:
bin_to_cat = {0:'LGBT',
 1:'alternative lifestyle',
 2:'book clubs',
 3:'career/business',
 4:'cars/motorcycles',
 5:'community/environment',
 6:'dancing',
 7:'education/learning',
 8:'fashion/beauty',
 9:'arts/culture',
 10:'fitness',
 11:'food/drink',
 12:'games',
 13:'health/wellbeing',
 14:'hobbies/crafts',
 15:'language/ethnic identity',
 16:'movements/politics',
 17:'movies/film',
 18:'music',
 19:'new age/spirituality',
 20:'outdoors/adventure',
 21:'paranormal',
 22:'parents/family',
 23:'pets/animals',
 24:'photography',
 25:'religion/beliefs',
 26:'sci-fi/fantasy',
 27:'singles',
 28:'socializing',
 29:'sports/recreation',
 30:'support',
 31:'tech',
 32:'writing'}

In [6]:
for key,value in bin_to_cat.items():
    print(meet_to_curl[value])

https://curlie.org/Society/Gay%2C_Lesbian%2C_and_Bisexual/
https://curlie.org/Society/Lifestyle_Choices/
https://curlie.org/Reference/Books
https://curlie.org/Business/
https://curlie.org/Business/Automotive/
https://curlie.org/Society/Issues/Environment/Organizations/
https://curlie.org/Arts/Performing_Arts/Dance
https://curlie.org/Reference/Education/
['https://curlie.org/Business/Design/Fashion', 'https://curlie.org/Shopping/Health/Beauty']
https://curlie.org/Arts/
https://curlie.org/Health/Fitness/
https://curlie.org/Recreation/Food/
https://curlie.org/Games/
https://curlie.org/Health/
https://curlie.org/Shopping/Crafts/
https://curlie.org/World/
https://curlie.org/Society/Politics/
https://curlie.org/Arts/Movies/
https://curlie.org/Arts/Music/
https://curlie.org/Society/Religion_and_Spirituality/New_Age/
https://curlie.org/Recreation/Outdoors/
https://curlie.org/Society/Paranormal/
https://curlie.org/Home/Family/
https://curlie.org/Recreation/Pets/
https://curlie.org/Arts/Photogra

# Context

## Parsing

In [7]:
# Function to map message
def internal_repr(message):
    j_mess = copy.deepcopy(message) #json.loads(message)
    j_event = j_mess['event']
    j_group = j_mess['group']
    j_topics = j_group['group_topics']
    
    for elem in j_topics:
        elem['@id'] = elem.pop('urlkey')
        elem['name'] = elem.pop('topic_name') # urlkey as @id, topic_name as name
    
    representation = {  # event
        '@type':'Event',
        '@id' : str(j_event['event_id']),
        'name': j_event['event_name'],
        'url' : j_event['event_url'],
        'startDate': j_event['time'],
        'organizer': {
            '@type':'Group',
            '@id': str(j_group['group_urlname']), # group
            'name':j_group['group_name'],
            'geo':{
                '@type':'Place',
                'latitude':j_group['group_lat'],
                'longitude': j_group['group_lon'],
                'city': j_group['group_city'],
                'country':j_group['group_country']
            },
            'url':"https://www.meetup.com/"+str(j_group['group_urlname'])+"/",
            'topics': j_topics 
        }
    }
    try:
        j_venue = j_mess['venue'] #venue
        representation['location'] = {
            '@type':'Place',
            '@id': str(j_venue['venue_id']),
            'name':j_venue['venue_name'],
            'latitude':j_venue['lat'],
            'longitude':j_venue['lon']
        }
        return representation
    except:
        return representation

In [8]:
def message_repr(content):
    contxt_repr = {
        "@version" : 1.1,
        "@base" : "https://www.meetology.onto/",
        "schema" : "https://schema.org/", 
        "sc" : "http://rdfs.org/sioc/spec/#term_",
        "wdt" : "https://www.wikidata.org/wiki/Property:",
        "xsd" : "https://www.w3.org/2001/XMLSchema#",
        "wtr" : "https://www.weatherforecast.org/",
        "cff" : "https://www.w3.org/2005/Incubator/ssn/ssnx/cf/cf-feature#",
        "seas" : "http://w3id.org/seas/",
        "name":{
            "@id":"schema:name",
            "@type":"xsd:literal"
        },
        "_key":{
            "@id":"schema:identifier",
            "@type":"xsd:literal"
        },
        "url":{
            "@id":"schema:url",
            "@type":"@id"
        },
        "startDate":{
            "@id":"schema:startDate",
            "@type":"xsd:dateTime"
        },
        "location":{
            "@id":"schema:location",
            "@type":"@id",
            "@context":{
                "@base":"/rdf/location/"
            }
        },
        "latitude":{
            "@id":"schema:latitude",
            "@type":"xsd:float"
        },
        "longitude":{
            "@id":"schema:longitude",
            "@type":"xsd:float"
        },
        "country":{
            "@id":"schema:Country",
            "@type":"xsd:literal"
        },
        "state":{
            "@id":"schema:State",
            "@type":"xsd:literal"
        },
        "city":{
            "@id":"schema:City",
            "@type":"xsd:literal"
        },
        "organizer":{
            "@id":"schema:organizer",
            "@type":"@id",
            "@context":{
                "@base":"/rdf/group/"
            }
        },
        "Group":{
            "@id":"sc:Community",
            "@type":"@id"
        },
        "Event":{
            "@id":"schema:Event",
            "@type":"@id"
        },
        "topics":{
            "@id":"sc:topic",
            "@type":"@id",
            "@context":{
                "@base":"/rdf/topics/"
            }
        },
        "Place":{
            "@id":"schema:Place",
            "@type":"@id"
        },
        "geo":{
            "@id":"schema:geo",
            "@type":"@id",
            "@context":{
                "@base":"/rdf/geo/"
            }
        },
        "pop":{
            "@id":"wdt:P1082",
            "@type":"xsd:float"
        },
        "elevation":{
            "@id":"wdt:P2044",
            "@type":"xsd:float"
        },
        "area":{
            "@id":"wdt:P2046",
            "@type":"xsd:float"
        },
        "Forecast":{
            "@id":"seas:Forecast",
            "@type":"@id"
        },
        "weather":{
            "@id":"wtr:weather",
            "@type":"@id",
            "@context":{
                "@base":"/rdf/weather/"
            }
        },
        "tempCelsius":{
            "@id":"cff:Temperature",
            "@type":"@id",
            "@context":{
                "@base":"/rdf/temperature/"
            }
        },
        "temp":{
            "@id":"wtr:temperatureForecast",
            "@type":"xsd:float"
        },
        "temp_min":{
            "@id":"wtr:temperatureMinForecast",
            "@type":"xsd:float"
        },
        "temp_max":{
            "@id":"wtr:temperatureMaxForecast",
            "@type":"xsd:float"
        },
        "status":{
            "@id":"wtr:status",
            "@type":"xsd:literal"
        },
        "humidityPercentage":{
            "@id":"cff:Humidity",
            "@type":"xsd:float"
        },
        "pressure":{
            "@id":"wtr:pressure",
            "@type":"xsd:float"
        },
        "windSpeed":{
            "@id":"cff:Wind",
            "@type":"xsd:float"
        },
        "rainVolume":{
            "@id":"wtr:rain",
            "@type":"xsd:float"
        },
        "cloudsCoverage":{
            "@id":"wtr:clouds",
            "@type":"xsd:float"
        },
        "snowVolume":{
            "@id":"wtr:snow",
            "@type":"xsd:float"
        },
        "description":{
            "@id":"schema:description",
            "@language":"",
            "@type":"xsd:literal"
        },
        "forecastDate":{
            "@id":"schema:temporal",
            "@type":"xsd:dateTime"
        },
        "meetup_category":{
            "@id":"schema:category",
            "@type":"xsd:literal"
        },
        "dmoz_category":{
            "@id":"schema:category",
            "@type":"xsd:literal"
        }
}
    
    nested = {}
    nested['@context'] = contxt_repr
    #results = copy.deepcopy(context)
    for key, value in internal_repr(content).items():
        nested[key] = value
        #results[key] = value
    return nested

## Enrich geonameid

In [9]:
def enrich_geonameid(parsed):
    message  = copy.deepcopy(parsed)
    conn = http.client.HTTPConnection('149.132.176.50', port = 9200)
    headers = {
         'Content-Type': "application/json",
         'cache-control': "no-cache",
         'Postman-Token': "f4e07e24-eeb6-4cb6-a6a8-bc3254f41192"
         }
    name = message['organizer']['geo']['city'].encode('utf-8').decode('unicode-escape')
    lat = message['organizer']['geo']['latitude']
    if message['organizer']['geo']['city'] == 'London':
        lon = -0.12
        lat = 51.5
    else:
        lon = message['organizer']['geo']['longitude']
    #print(name, lat, lon)
    #break
    payload = '{"query": {"bool": \
                            {"minimum_should_match": 1,"should": \
                             [{"match": {"name": {"query":"'+name+'"}}}, \
                              {"match": {"alternatenames":"'+name+'"}}, \
                              {"match": {"asciiname": {"query":"'+name+'"} }}], \
                             "filter": [{"term": {"fclass":"A" }}, \
                                        {"geo_distance" : \
                                         {"distance" :"5km","location" :{"lat" : '+str(lat)+', "lon" : '+str(lon)+'}}}]}}}'

    conn.request("GET", "geonames/geoname/_search", payload, headers)

    res = conn.getresponse()
    data = res.read()
    output = json.loads(data.decode("utf-8"))
    try:
        if message['organizer']['geo']['city'] == 'Paris':
            message['organizer']['geo']['pop'] = 2206488
            message['organizer']['geo']['@id'] = str(2968815)
        else:
            pop = output['hits']['hits'][0]['_source']['population']
            geo = output['hits']['hits'][0]['_source']['geonameid']
            message['organizer']['geo']['pop'] = pop
            message['organizer']['geo']['@id'] = str(geo)
        #success += 1
        #counter += 1
    except:
        payload = '{"query": {"bool": \
                    {"minimum_should_match": 1,"should": \
                    [{"match": {"name": {"query":"'+name+'"}}}, \
                    {"match": {"alternatenames":"'+name+'"}}, \
                    {"match": {"asciiname": {"query":"'+name+'"} }}], \
                    "filter": [{"term": {"fclass":"P" }}, \
                    {"geo_distance" : \
                    {"distance" :"5km","location" :{"lat" : '+str(lat)+', "lon" : '+str(lon)+'}}}]}}}'
        conn.request("GET", "geonames/geoname/_search", payload, headers)

        res = conn.getresponse()
        data = res.read()
        output = json.loads(data.decode("utf-8"))
       # count += 1
        try:
            pop = output['hits']['hits'][0]['_source']['population']
            geo = output['hits']['hits'][0]['_source']['geonameid']
            message['organizer']['geo']['pop'] = pop
            message['organizer']['geo']['@id'] = str(geo)
#             success += 1
#             counter += 1
        except Exception as e:
            print(e, file = sys.stderr)
            pass
#             failure += 1
#             counter += 1
    return message

In [10]:
# conn = http.client.HTTPConnection('149.132.176.50', port = 9200)
# headers = {
#      'Content-Type': "application/json",
#      'cache-control': "no-cache",
#      'Postman-Token': "f4e07e24-eeb6-4cb6-a6a8-bc3254f41192"
#      }
# payload = '{"query": {"bool": \
#             {"minimum_should_match": 1,"should": \
#             [{"match": {"name": {"query":"London"}}}, \
#             {"match": {"alternatenames":"London"}}, \
#             {"match": {"asciiname": {"query":"London"} }}], \
#             "filter": [{"term": {"fclass":"A" }}, \
#             {"geo_distance" : \
#             {"distance" :"10km","location" :{"lat" : 51.5, "lon" : -0.02}}}]}}}'
# conn.request("GET", "geonames/geoname/_search", payload, headers)

# res = conn.getresponse()
# data = res.read()
# output = json.loads(data.decode("utf-8"))
# pp(output)

## Enrich demography (wikidata)

In [11]:
def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [12]:
def enrich_demography(enriched):
    message = copy.deepcopy(enriched)
    endpoint_url = "https://query.wikidata.org/sparql"
    try:
        geonameid = message['organizer']['geo']['@id']
        query = """SELECT ?city ?pop ?area ?elevation WHERE {
          ?city wdt:P1566 """+'"'+str(geonameid)+'"'+""".
          OPTIONAL { ?city wdt:P1082 ?pop. }
          OPTIONAL { ?city wdt:P2046 ?area. }
          OPTIONAL { ?city wdt:P2044 ?elevation. }
        }
        limit 1"""

        proxy_support = urllib.request.ProxyHandler({'http':'127.0.0.1'})
        opener = urllib.request.build_opener(proxy_support)
        urllib.request.install_opener(opener)

        flag = False
        counter = 0
        max_count = 20
        while (not flag) and (counter <= max_count):
            try:
                results = get_results(endpoint_url, query)
                flag = True
            except:
                counter += 1
                pass    

        try:
            results = get_results(endpoint_url, query)
            #success += 1
            try:
                message['organizer']['geo']['url'] = results['results']['bindings'][0]['city']['value']
                #switch_1 = True
            except:
                #switch_1 = False
                pass
            try:
                message['organizer']['geo']['area'] = results['results']['bindings'][0]['area']['value']
                #switch_2 = True
            except:
                #switch_2 = False
                pass
            try:
                message['organizer']['geo']['elevation'] = results['results']['bindings'][0]['elevation']['value']
                #switch_3 = True
            except:
                #switch_3 = False
                pass
            # don't subscribe population from geonameid with wikidata
            if message['organizer']['geo']['pop'] == 0:
                try:
                    message['organizer']['geo']['pop'] = results['results']['bindings'][0]['pop']['value']
                    #switch_4 = True
                except:
                    #switch_4 = False
                    pass

#             debug = switch_1+switch_2+switch_3+switch_4
#             if debug >= 1 and debug <= 3:
#                 partial += 1

        except Exception as e:
            # print(e)
#             failure += 1
             pass

#         count += 1
#         time.sleep(0.1)
    
    except KeyError:
        pass
   # except Exception as ex:
        #print(ex)
    
    return message

## Enrich weather (OpenWeather) 

In [13]:
api_key1 = 'a9eeefd106f01415cea80f4de6ec7178'
api_key2 = '471988b361b2928da5b1f4917df0b609'
api_key3 = '1da352c2830f5905b9c9c9d75b0b023a'
def enrich_weather(demography):
    message = copy.deepcopy(demography)
    owm = pyowm.OWM(api_key2)
    if message['startDate'] <= time.time()*1000 + 388800000 and  message['startDate'] >= time.time()*1000 + 10800000:
        message['weather'] = {"@type":"Forecast"} # init section
        message['weather']['forecastDate'] = dt.utcnow().strftime('%Y-%m-%d %H:%M:%S+00')
        try:
            obs_fs = owm.three_hours_forecast_at_coords(message['organizer']['geo']['latitude'],
                                                        message['organizer']['geo']['longitude'])
            w = obs_fs.get_weather_at(str(dt.fromtimestamp(message['startDate']/1000))+'+00')
            #print(w.to_JSON())
            #break
            #success +=1
            try:
                message['weather']['tempCelsius'] = {'@type':'Forecast',
                                                     'temp': w.get_temperature('celsius')['temp'],
                                                     'temp_max': w.get_temperature('celsius')['temp_max'],
                                                     'temp_min': w.get_temperature('celsius')['temp_min']}
            except Exception as e:
                print(e)
                pass
            try:
                 message['weather']['status'] = w.get_status()
            except:
                pass
            try:
                message['weather']['humidityPercentage'] = w.get_humidity()
            except:
                pass
            try:
                message['weather']['pressure'] = w.get_pressure()['press']
            except:
                pass
            try:
                message['weather']['windSpeed'] = w.get_wind()['speed']
            except:
                pass
            try:
                message['weather']['rainVolume'] = w.get_rain()
            except:
                pass
            try:
                 message['weather']['cloudsCoverage'] = w.get_clouds()
            except:
                pass
            try:
                 message['weather']['snowVolume'] = w.get_snow()
            except:
                pass
                
        except Exception as e:
            #print("{} at line {}".format(e, count))
            print(e)
            #failure += 1
            pass
        
        #counter += 1
        #time.sleep(1.05)
    
    
    try:
        date_time = dt.fromtimestamp(message['startDate']/1000).strftime('%Y-%m-%d %H:%M:%S+00')
        message['startDate'] = date_time
    except Exception as ex:
        #print("Failed timestamp conversion at line {} beacause of {}".format(count, ex))
        pass
    
    
    return message

## Enrich Event description (rest API Meetup)

In [14]:
def enrich_description(weather):
    message = copy.deepcopy(weather)
    id_0 = message['@id']
    max_key = "445811b5a6b424f7e79342826176d"
    fabri_key = "2e12625a12642d6ac743d19566c393e"
    per_page = 1
    offset = 0
    params = {"event_id":id_0, "key":fabri_key, "page":per_page, "offset":offset}
    try:
        request = requests.get("http://api.meetup.com/2/events", params = params)

        data = request.json()
        #print(data)
        desc = data["results"][0]["description"]
        soup = BeautifulSoup(desc, "lxml")
        clean = soup.get_text()
        message['description'] = clean
        message['@context']['description']['@language'] = detect(clean)
    except:
        pass
    
    return message

## Meetup Producer (websocket streaming from ws://stream.meetup.com/2/rsvps)

In [15]:
from cleansing import *
import joblib
doc2vec_model=joblib.load("../classifier/joblib/model_pair_all_10_ep_Porter_no_badwords.joblib")

rf_model=joblib.load("../classifier/joblib/model_rf_pair.joblib")

def import_arango(message):
    message.update({'_key': message['@id']})
    try:
        metadata = messages.insert(message)
        assert metadata['_id'] == 'messages/{}'.format(message['_key'])
        assert metadata['_key'] == '{}'.format(message['_key'])
    except:
        pass
    

def on_message(ws, message):
    message = json.loads(message)
    id_ = message['event']['event_id']
    if id_ not in black_list and message['group']['group_country'] in white_list:
        parsed = message_repr(message)
        enriched = enrich_geonameid(parsed)
        demo = enrich_demography(enriched)
        weather = enrich_weather(demo)
        described = enrich_description(weather)
        if "description" in described.keys():
            desc=described["description"]
            clean_desc=clean_description(desc)
            tag_doc=fake_tagged_doc(clean_desc)
            desc_vec=vec_for_learning(doc2vec_model, tag_doc)
            predicted_category=rf_model.predict(desc_vec)
            #print(desc, predicted_category)
            described["meetup_category"]=predicted_category[0]
            described["dmoz_category"]=dmoz_cat[predicted_category[0]]
            #print(described["meetup_category"],described["dmoz_category"])
        container.append(described)
        black_list.append(id_)
        if len(container)%10 == 0:
            end = time.time()
            operation_time = end - start
            operation_time = round(operation_time, 2)
            if operation_time <= 60:
                print("{} messages stored in {} s".format(len(container), str(operation_time)), file = sys.stderr)
            else:
                print("{} messages stored in {} m".format(len(container), str(round(operation_time/60, 2))), file = sys.stderr)
                
        import_arango(described)
        if len(container) >= n_of_messages:
            ws.keep_running = False


def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###", file = sys.stderr)
    print("Taken {} messages".format(len(container)), file = sys.stderr)

In [16]:
websocket.enableTrace(True)
client = ArangoClient(protocol='http', host='10.9.13.9', port=7474)
sys_db = client.db('_system', username='root', password='root')
if not sys_db.has_database('meetology'):
    sys_db.create_database('meetology')
db = client.db('meetology', username='root', password='root')

name_collection = "messing_with_cat"
if db.has_collection(name_collection):
    messages = db.collection(name_collection)
else:
    messages = db.create_collection(name_collection)

#messages.truncate()    # NB DECOMMENTA SOLO SE INTENDI RE-INIZIALIZZARE IL DB

In [25]:
n_of_messages = 100
container = []
black_list = []
white_list = ['it', 'fr', 'es', 'de', 'gb', 'pt', 'fi', 'sw', 'nt']
start = time.time()
while len(container) < n_of_messages:
    ws = websocket.WebSocketApp("ws://stream.meetup.com/2/rsvps",
                                on_message = on_message,
                                on_error = on_error,
                                on_close = on_close)
    
    
    ws.run_forever()
    break

--- request header ---
GET /2/rsvps HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: stream.meetup.com
Origin: http://stream.meetup.com
Sec-WebSocket-Key: ZrbxWe46nFz8w9ZQWAxZZw==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Thu, 11 Jul 2019 09:12:57 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: B1U6Mdqft9a863MrvaddSaTUP+U=
-----------------------
100%|██████████| 1/1 [00:00<00:00, 102.15it/s]
Detector is not able to detect the language reliably.
100%|██████████| 1/1 [00:00<00:00, 106.38it/s]
10 messages stored in 29.72 s
100%|██████████| 1/1 [00:00<00:00, 71.37it/s]
20 messages stored in 1.44 m
100%|██████████| 1/1 [00:00<00:00, 208.74it/s]
30 messages stored in 1.86 m
100%|██████████| 1/1 [00:00<00:00, 115.73it/s]
/home/dario/.local/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "https://ti.to/circular-economy-club-london/the-year-2029-circular-economy-iot-and-meeting-recyclin

 -- https://ti.to/circular-economy-club-london/the-year-2029-circular-economy-iot-and-meeting-recycling-targets


100%|██████████| 1/1 [00:00<00:00, 111.84it/s]
Detector is not able to detect the language reliably.
100%|██████████| 1/1 [00:00<00:00, 50.25it/s]
50 messages stored in 2.65 m
100%|██████████| 1/1 [00:00<00:00, 129.71it/s]
60 messages stored in 2.97 m
100%|██████████| 1/1 [00:00<00:00, 114.76it/s]
list index out of range
100%|██████████| 1/1 [00:00<00:00, 76.08it/s]
70 messages stored in 3.43 m
100%|██████████| 1/1 [00:00<00:00, 92.66it/s]
80 messages stored in 3.83 m
100%|██████████| 1/1 [00:00<00:00, 99.04it/s]
Detector is not able to detect the language reliably.
100%|██████████| 1/1 [00:00<00:00, 89.09it/s]
90 messages stored in 4.41 m
100%|██████████| 1/1 [00:00<00:00, 57.79it/s]
send: b'\x88\x82<H\x14\x0b?\xa0'


### closed ###
Taken 95 messages


## Retake data from ArangoDB

In [17]:
aql = db.aql
start = time.time()
cursor = db.aql.execute(
  'FOR doc IN messing_with_cat RETURN doc'
)
data_mess = [doc for doc in cursor]
print("time {}".format(time.time()-start))

time 0.3536999225616455


## Parse from JSON-LD to RDF

### Using rdflib-jsonld plugin

In [18]:
rdf_lib = Graph() # using directly rdflib-jsonld

In [19]:
# parsed['@context']['@version'] = "1.1"
# rdf_lib.parse(data = json.dumps(parsed), format = "json-ld")
for elem in data_mess:
    elem['@context']['@version'] = "1.1"
    rdf_lib.parse(data = json.dumps(elem), format = "json-ld")

In [20]:
[p for p in rdf_lib.namespaces()]

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('schema', rdflib.term.URIRef('https://schema.org/')),
 ('wdt', rdflib.term.URIRef('https://www.wikidata.org/wiki/Property:')),
 ('xsd1', rdflib.term.URIRef('https://www.w3.org/2001/XMLSchema#')),
 ('wtr', rdflib.term.URIRef('https://www.weatherforecast.org/')),
 ('cff',
  rdflib.term.URIRef('https://www.w3.org/2005/Incubator/ssn/ssnx/cf/cf-feature#')),
 ('seas', rdflib.term.URIRef('http://w3id.org/seas/'))]

In [21]:
result_query = rdf_lib.query("""SELECT ?m ?n WHERE{?e schema:location ?l;
                                                schema:name ?m .
                                            ?l schema:name ?n .}""")

In [22]:
for row in result_query:
    #print(row)
    print("%s IN LOCATION %s" % row)

Be Our Guest! Please join us for our twice monthly meeting IN LOCATION Clock Warehouse
Cycle & dine - Goat cheesery Capriolenhof** IN LOCATION Berlin Ostkreuz - platform 13/14
Lectures de la fin du monde IN LOCATION Madame Bovary
#24 Let’s play! - Die agile Welt spielerisch Kennenlernen  IN LOCATION AOE GmbH
Google G Suite Anwendertraining IN LOCATION //SEIBERT/MEDIA
Google AI Workshop: Machine learning with Tensorflow IN LOCATION Central Saint Giles
Initiation to Belgina beers IN LOCATION Lambicus Bar
WORKSHOP | Cultivating success | Brighton IN LOCATION Propellernet
Meet Amazing Data Women - Summer Social! IN LOCATION Tech Incubator
GO: Badminton Night (Stourbridge) IN LOCATION Crystals leisure centre
Early Birds Toastmasters – Weekly Meeting for Public Speaking IN LOCATION Freemasons' Hall
The State of AI — Successes & Limitations IN LOCATION ThoughtWorks Cologne
Free Terrible Football @ Tooting Bec Common Saturday 10:30AM IN LOCATION Tooting Bec Common (Field just north of Bedford 

In [23]:
for s,p,o in py_ld:
    print("{} ---> {} ---> {}".format(s,p,o))

NameError: name 'py_ld' is not defined

In [24]:
data_mess[0]

{'_key': 'lqdfpqyzkbvb',
 '_id': 'messing_with_cat/lqdfpqyzkbvb',
 '_rev': '_Y8_MqqK---',
 '@context': {'@version': '1.1',
  '@base': 'https://www.meetology.onto/',
  'schema': 'https://schema.org/',
  'sc': 'http://rdfs.org/sioc/spec/#term_',
  'wdt': 'https://www.wikidata.org/wiki/Property:',
  'xsd': 'https://www.w3.org/2001/XMLSchema#',
  'wtr': 'https://www.weatherforecast.org/',
  'cff': 'https://www.w3.org/2005/Incubator/ssn/ssnx/cf/cf-feature#',
  'seas': 'http://w3id.org/seas/',
  'name': {'@id': 'schema:name', '@type': 'xsd:literal'},
  '_key': {'@id': 'schema:identifier', '@type': 'xsd:literal'},
  'url': {'@id': 'schema:url', '@type': '@id'},
  'startDate': {'@id': 'schema:startDate', '@type': 'xsd:dateTime'},
  'location': {'@id': 'schema:location',
   '@type': '@id',
   '@context': {'@base': '/rdf/location/'}},
  'latitude': {'@id': 'schema:latitude', '@type': 'xsd:float'},
  'longitude': {'@id': 'schema:longitude', '@type': 'xsd:float'},
  'country': {'@id': 'schema:Coun

In [33]:
result_more_query = rdf_lib.query(
    """
        SELECT ?name ?cat ?country
        WHERE{
            ?event schema:name ?name;
            schema:organizer ?org;
            schema:category ?cat FILTER ( str(?cat) = "tech" ).
            ?org schema:geo ?geo.
            ?geo mty:countryname ?country  FILTER ( str(?country) = "gb" ).          
        }
    """
)

for row in result_more_query:
    print("%s ---Has Category---> %s --Country-->%s" % row)

#NottsTest - TBA ---Has Category---> tech --Country-->gb
Google AI Workshop: Machine learning with Tensorflow ---Has Category---> tech --Country-->gb
Hands-on Meetup: Create a design system in React with Styled System -TICKET only ---Has Category---> tech --Country-->gb
Reading SEO- On-Page Technical, Power of Information for SEO & Internal Search  ---Has Category---> tech --Country-->gb
Emily Jiang- On Stage Hacking: Build 12-Factor Microservices in an Hour ---Has Category---> tech --Country-->gb
Community meetup ---Has Category---> tech --Country-->gb
Agile HR Massive Morning Meetup | London | Fri 13th Sept ---Has Category---> tech --Country-->gb
 Marc Gravell - gRPC in .NET - Hosted by IRESS ---Has Category---> tech --Country-->gb
July Gophers ---Has Category---> tech --Country-->gb
Meet Amazing Data Women - Summer Social! ---Has Category---> tech --Country-->gb
Keynote by Angela Yu on Why I'm Building My Next App in Flutter ---Has Category---> tech --Country-->gb
London #PowerBI us

### Using pyld

In [60]:
py_ld = Graph()
#py_ld.open("store", create = True)
for elem in data:
    elem['@context']['@version'] = 1.1
    elem['@context']['cf'] = "https://purl.oclc.org/NET/ssnx/qu/dim#/"
    nquads = jsonld.to_rdf(elem, {'format':'application/n-quads','produceGeneralizedRdf':True})
    py_ld.parse(data = nquads, format = "nquads")

In [61]:
schema = Namespace("https://schema.org/")
sc = Namespace("https://rdfs.org/sioc/spec/#term_topic/")
wdt = Namespace("https://www.wikidata.org/wiki/Property:")
wtr = Namespace("https://www.weatherforecast.org/")
#cf = Namespace("https://purl.oclc.org/NET/ssnx/qu/dim#/")
cff = Namespace("https://purl.oclc.org/NET/ssnx/cf/cf-feature#/")
seas = Namespace("https://w3id.org/seas/")

py_ld.namespace_manager.bind("schema", schema)
py_ld.namespace_manager.bind("sc", sc)
py_ld.namespace_manager.bind("wdt", wdt)
py_ld.namespace_manager.bind("wtr", wtr)
#py_ld.namespace_manager.bind("cf", cf)
py_ld.namespace_manager.bind("cff", cff)
py_ld.namespace_manager.bind("seas", seas)

In [62]:
[print(p) for p in py_ld.namespaces()]

('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'))
('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'))
('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#'))
('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#'))
('schema', rdflib.term.URIRef('https://schema.org/'))
('sc', rdflib.term.URIRef('https://rdfs.org/sioc/spec/#term_topic/'))
('wdt', rdflib.term.URIRef('https://www.wikidata.org/wiki/Property:'))
('wtr', rdflib.term.URIRef('https://www.weatherforecast.org/'))
('cf', rdflib.term.URIRef('https://purl.oclc.org/NET/ssnx/qu/dim#/'))
('cff', rdflib.term.URIRef('https://purl.oclc.org/NET/ssnx/cf/cf-feature#/'))
('seas', rdflib.term.URIRef('https://w3id.org/seas/'))


[None, None, None, None, None, None, None, None, None, None, None]

In [51]:
pp(data[2])

{'@context': {'@base': 'https://www.meetology.onto/',
              '@version': 1.1,
              'Event': {'@id': 'schema:Event', '@type': '@id'},
              'Forecast': {'@id': 'seas:Forecast', '@type': '@id'},
              'Group': {'@id': 'sc:community', '@type': '@id'},
              'Place': {'@id': 'schema:Place', '@type': '@id'},
              '_key': {'@id': 'schema:identifier', '@type': 'xsd:literal'},
              'area': {'@id': 'wdt:P2046', '@type': 'xsd:float'},
              'cf': 'http://purl.oclc.org/NET/ssnx/qu/dim#/',
              'cff': 'http://purl.oclc.org/NET/ssnx/cf/cf-feature#',
              'city': {'@id': 'schema:City', '@type': 'xsd:literal'},
              'cloudsCoverage': {'@id': 'wtr:clouds', '@type': 'xsd:float'},
              'country': {'@id': 'schema:Country', '@type': 'xsd:literal'},
              'description': {'@id': 'schema:description',
                              '@language': 'es',
                              '@type': 'xsd:literal

In [63]:
# da controllare il la temperatura per quanto riguarda la query
result_query = py_ld.query("""SELECT  ?f  WHERE{?e cf:temperature ?t .
                                                    ?t wtr:temperatureForecast ?f .}""")

In [64]:
for row in result_query:
    #print(row)
    print("%s" % row)

3.175E1
1.995E1


In [123]:
for s,p,o in py_ld:
    print("{} ---> {} ---> {}".format(s,p,o))

https://www.meetology.onto/mlmkwqyzkbhb ---> http://www.w3.org/1999/02/22-rdf-syntax-ns#type ---> https://schema.org/Event
https://www.meetology.onto/rdf/group/Enjoy-Barcelona-Catalonia ---> http://rdfs.org/sioc/spec/#term_topic/topic ---> https://www.meetology.onto/rdf/topics/fun-days-out
https://www.meetology.onto/rdf/group/Yoga-Playa-Poble-Nou ---> http://rdfs.org/sioc/spec/#term_topic/topic ---> https://www.meetology.onto/rdf/topics/healthy-living
N55317207cefe426bb7443e291cd1d9eb ---> http://purl.oclc.org/NET/ssnx/cf/cf-feature#Humidity ---> 51
https://www.meetology.onto/rdf/group/meetup-group-CPBTWGxA ---> http://rdfs.org/sioc/spec/#term_topic/topic ---> https://www.meetology.onto/rdf/topics/environmental-education
https://www.meetology.onto/rdf/location/26348528 ---> https://schema.org/longitude ---> -1.467759E0
https://www.meetology.onto/rdf/group/the-natural-yogi ---> https://schema.org/geo ---> https://www.meetology.onto/rdf/geo/3117735
https://www.meetology.onto/rdf/topics/e

## Description translation (improve RandomForest performance)

In [109]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])

    return clean_text

# translator = Translator()
# give_emoji_free_text(container[90]['description'])
# translator.translate(give_emoji_free_text(container[90]['description']), dest = 'en').text